<a href="https://colab.research.google.com/github/andrew-ryabchenko/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:

DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'


In [ ]:
import pandas as pd


# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)


In [ ]:
# Converting strings to numeric types

# Identifiying odd entries
df.sort_values(by = 'LAND_SQUARE_FEET', ascending = True)['LAND_SQUARE_FEET']

# Dropping odd values
df = df[~(df['LAND_SQUARE_FEET'] == '########')]

# Dropping NaN values
df.dropna(subset = ['LAND_SQUARE_FEET'], inplace = True)

# Verifiying the result
df.sort_values(by = 'LAND_SQUARE_FEET', ascending = True)['LAND_SQUARE_FEET']

16917      0
6226       0
16010      0
16011      0
16012      0
        ... 
823      990
17206    992
17205    992
17204    992
7030     997
Name: LAND_SQUARE_FEET, Length: 22986, dtype: object

In [ ]:
# Removing delimiters and spaces and coverting to numeric type

df['LAND_SQUARE_FEET'] = (
    df['LAND_SQUARE_FEET']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)


In [ ]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [ ]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [ ]:
df['NEIGHBORHOOD'].unique()

array(['OTHER', 'UPPER EAST SIDE (59-79)', 'UPPER EAST SIDE (79-96)',
       'BOROUGH PARK', 'ASTORIA', 'FOREST HILLS',
       'UPPER WEST SIDE (59-79)', 'BEDFORD STUYVESANT', 'EAST NEW YORK',
       'FLUSHING-NORTH', 'GRAMERCY'], dtype=object)

In [ ]:
# Filtering data to only buildings of '01 ONE FAMILY DWELLINGS' class

mask = df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS'
df = df[mask]

In [ ]:
# Dropping useless features for our model

df.drop(columns=['BUILDING_CLASS_CATEGORY','ZIP_CODE', 'EASE-MENT', 'ADDRESS', 'APARTMENT_NUMBER'], inplace=True)

In [ ]:
df.drop(columns=['LOT', 'BLOCK', 'BUILDING_CLASS_AT_PRESENT','TAX_CLASS_AT_PRESENT'], inplace = True)

In [ ]:
# Final filtering of NaN values

df.dropna()

,BOROUGH,NEIGHBORHOOD,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
7,2,OTHER,1.0,0.0,1.0,3404,1328.0,1925.0,1,A1,0,01/01/2019
8,2,OTHER,1.0,0.0,1.0,2042,1728.0,1935.0,1,A5,0,01/01/2019
9,2,OTHER,1.0,0.0,1.0,2042,1728.0,1935.0,1,A5,0,01/01/2019
42,3,OTHER,1.0,0.0,1.0,3333,1262.0,1925.0,1,A1,0,01/01/2019
44,3,OTHER,1.0,0.0,1.0,6800,1325.0,1930.0,1,A9,550000,01/01/2019
...,...,...,...,...,...,...,...,...,...,...,...,...
23029,4,OTHER,1.0,0.0,1.0,3300,1478.0,1925.0,1,A2,635000,04/30/2019
23031,4,OTHER,1.0,0.0,1.0,2400,1280.0,1930.0,1,A1,514000,04/30/2019
23032,4,OTHER,1.0,0.0,1.0,4000,1333.0,1945.0,1,A0,635000,04/30/2019
23033,4,OTHER,1.0,0.0,1.0,2500,1020.0,1935.0,1,A1,545000,04/30/2019


In [ ]:
# Removing outlier prices
filter = (df['SALE_PRICE'] >= 100000) & (df['SALE_PRICE'] <= 20000000)
df = df[filter]

In [ ]:
# Splitting dataset into test/train variants

In [ ]:
df['SALE_DATE'] = df['SALE_DATE'].apply(lambda x: pd.to_datetime(x))

In [ ]:
cutoff = df['SALE_DATE'] <= pd.to_datetime('2019-03-31')

In [ ]:
train_data = df[cutoff].set_index('SALE_DATE')
test_data = df[~cutoff].set_index('SALE_DATE')

In [ ]:
train_data.nunique()

BOROUGH                             5
NEIGHBORHOOD                        8
RESIDENTIAL_UNITS                   2
COMMERCIAL_UNITS                    3
TOTAL_UNITS                         4
LAND_SQUARE_FEET                  921
GROSS_SQUARE_FEET                 967
YEAR_BUILT                         86
TAX_CLASS_AT_TIME_OF_SALE           1
BUILDING_CLASS_AT_TIME_OF_SALE     12
SALE_PRICE                        935
dtype: int64

In [ ]:
# One-Hot Encoding categorical variables

from category_encoders import OneHotEncoder

In [ ]:
enc = OneHotEncoder(use_cat_names=True)
enc.fit(train_data)
enc.fit(test_data)
X_test = enc.transform(test_data)
X_train = enc.transform(train_data);

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
# Creating target vectors

y_test = X_test[['SALE_PRICE']]
y_train = X_train[['SALE_PRICE']]

In [ ]:
X_test.drop(columns=['SALE_PRICE'], inplace=True)
X_train.drop(columns=['SALE_PRICE'], inplace=True)

In [ ]:
# Creating Linear and Ridge regression models

from sklearn.linear_model import LinearRegression, Ridge

model_lm = LinearRegression()
model_r = Ridge(alpha=1)
# Fitting models into train data

model_lm.fit(X_train, y_train)
model_r.fit(X_train, y_train)
# Making predictions

lm_pred = model_lm.predict(X_test)
r_pred = model_r.predict(X_test)

In [ ]:
# Calculating Baseline MAE

baseline = y_test['SALE_PRICE'].mean()

In [ ]:
# Comparing performance metrics

from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [ ]:
print(f'Baseline MAE: {mean_absolute_error(y_test, [baseline]*len(y_test))}')
print(f'LM MAE: {mean_absolute_error(y_test, lm_pred)}')
print(f'RIDGE MAE: {mean_absolute_error(y_test, r_pred)}')

Baseline MAE: 337003.2430238458
LM MAE: 216487.5791476408
RIDGE MAE: 221621.4684198869


In [ ]:
print(f'Baseline R^2: {r2_score(y_test, [baseline]*len(y_test))}')
print(f'LM R^2: {r2_score(y_test, lm_pred)}')
print(f'RIDGE R^2: {r2_score(y_test, r_pred)}')


Baseline R^2: 0.0
LM R^2: 0.7470742594850199
RIDGE R^2: 0.7369961178904021
